 # Preparing shapefiles from a given CSV file 

In [35]:
# Imports
import pandas as pd
import ee
import geemap
import random
import geopandas as gpd
from shapely.geometry import Point, Polygon
from fiona.crs import from_epsg
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Required package for using geopandas
!pip install xlrd

## Working with the local excel file

In [5]:
# Read the raw excel file. 
# The raw excel files contains a lot of redundant information in the heading and hence NaN or unnamed values will be
# displayed as we try to visualize it without any filter.
read_excel = pd.read_excel('assets/WUP2014-F12-Cities_Over_300K.xls',index_col=False)

In [7]:
# Filter out the columns we want to work on.
cols = ['Index','Country or area','Latitude','Longitude','Urban Agglomeration']

In [8]:
# Read the file from header (row no 16). The rows above 16th contains redundant information.
file = pd.read_excel('WUP2014-F12-Cities_Over_300K.xls',header=16,usecols=cols,index_col=False)

In [9]:
file

,Index,Country or area,Urban Agglomeration,Latitude,Longitude
0,1,Afghanistan,Herat,34.348170,62.199670
1,2,Afghanistan,Kabul,34.528887,69.172460
2,3,Afghanistan,Kandahar,31.613320,65.710130
3,4,Albania,Tiranë (Tirana),41.327500,19.818890
4,5,Algeria,Annaba,36.900000,7.766670
...,...,...,...,...,...
1687,1688,Zambia,Lusaka,-15.413374,28.277148
1688,1689,Zambia,Ndola,-12.958670,28.636590
1689,1690,Zimbabwe,Bulawayo,-20.150000,28.583330
1690,1691,Zimbabwe,Chitungwiza,-18.012740,31.075550


In [10]:
# We also require the Country == India rows for the above filtered dataframe. 
# This is the final dataframe we will be working with.

final_df = file.loc[file['Country or area'] == 'India']

In [11]:
final_df

,Index,Country or area,Urban Agglomeration,Latitude,Longitude
707,708,India,Agartala,23.836390,91.275000
708,709,India,Agra,27.183330,78.016670
709,710,India,Ahmadabad,23.033330,72.616670
710,711,India,Ahmadnagar,19.083330,74.733330
711,712,India,Aizawl,23.724440,92.717500
...,...,...,...,...,...
868,869,India,Vellore,12.933330,79.133330
869,870,India,Vijayawada,16.516670,80.616670
870,871,India,Visakhapatnam,17.681874,83.209685
871,872,India,Warangal,18.000000,79.583330


## Get the Map object as ipyleaflets

In [12]:
# Get the GEE map object
Map = geemap.Map(center=[40,100], zoom=4)
Map

Map(center=[40, 100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [15]:
# Create a hashmap to map the city center coordinates with the city name
city_dict = {}
cities = final_df['Urban Agglomeration'].tolist()
coor = list(zip(final_df['Longitude'].tolist(),final_df['Latitude'].tolist()))
shapefile_map = dict(zip(cities,coor))

In [18]:
# Take the first eight data 
sliced_map = dict(list(shapefile_map.items())[:len(shapefile_map)//20])
sliced_map

{'Agartala': (91.275, 23.83639),
 'Agra': (78.01667, 27.18333),
 'Ahmadabad': (72.61667, 23.03333),
 'Ahmadnagar': (74.73333, 19.08333),
 'Aizawl': (92.7175, 23.72444),
 'Ajmer': (74.63333, 26.45),
 'Akola': (77.0, 20.73333),
 'Aligarh': (78.08333, 27.88333)}

## Helper functions

In [30]:
def create_polygon(coordinates, polygon_name):
    ''' 
    Create a polygon from coordinates
    ''' 
    polygon = Polygon(coordinates)
    gdf = gpd.GeoDataFrame()
    gdf.loc[0,'name'] = polygon_name
    gdf.loc[0, 'geometry'] = polygon
    
    gdf.crs = from_epsg(4326)
    return gdf


## Loop over the maps

In [33]:
root_path = '166_city_shapefiles/'
if not os.path.exists(root_path):
        os.makedirs(root_path)

In [36]:
for city,shape in sliced_map.items():
      
    city_block = ee.Geometry.Point(shape).buffer(14400).bounds()
    pointCoordinates = city_block.coordinates()
    points = pointCoordinates.getInfo()[0]
    
    # create polygon shapefiles
    city_shape = create_polygon(points, city)
    
    curr_dir = root_path+ city + '/'
    
    if not os.path.exists(curr_dir):
        os.makedirs(curr_dir)
    
    # Export to shapefile if you want
    city_shape.to_file(curr_dir + city + '.shp')
    

## Test the shapefiles

In [81]:
test_sp_path = root_path + 'Agra/' + 'Agra.shp'
# test any shapefile and the data as a geopandas dataframe
data = gpd.read_file(test_sp_path)

# Print columns
print(data.columns)
print(data['name'].unique())

Index(['name', 'geometry'], dtype='object')
['Amsterdam']


In [37]:
Map2 = geemap.Map(center=[40,100], zoom=4)
Map2

Map(center=[40, 100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [38]:
# add the necessary shapefile
roi = geemap.shp_to_ee(test_sp_path)
Map2.addLayer(roi, {}, 'Agra')